In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import tensorflow as tf

# Tipy dla kolegi:
# 1. Będziesz potrzebował bibliotek: scikit-learn, matplotlib, seaborn
#    (pip install scikit-learn matplotlib seaborn)
# 2. Skup się na `sklearn.metrics` - tam jest wszystko, czego potrzebujesz.
# 3. Nie polegaj TYLKO na 'accuracy'. Przy okluzji ważne jest,
#    czy model nie faworyzuje jakiejś klasy. Dlatego `classification_report`
#    (z Precision, Recall, F1-Score) jest kluczowy.
# 4. Macierz pomyłek (Confusion Matrix) to Twój najlepszy przyjaciel.
#    Pokaże Ci wizualnie, z którymi osobami model ma największy problem.

def plot_confusion_matrix(y_true, y_pred, class_names):
    """
    Rysuje ładną macierz pomyłek.
    """
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(15, 15))
    sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Przewidziana etykieta (Predicted)')
    plt.ylabel('Prawdziwa etykieta (True)')
    plt.title('Macierz Pomyłek (Confusion Matrix)')
    # Zapisz do pliku, żeby pokazać na prezentacji
    plt.savefig('confusion_matrix.png') 
    print("Macierz pomyłek została zapisana do pliku 'confusion_matrix.png'")

def evaluate_model(model_path, test_generator):
    """
    Wczytuje wytrenowany model i ocenia go na zbiorze testowym.

    Argumenty:
    model_path (str): Ścieżka do zapisanego modelu (np. 'my_model.h5').
    test_generator (tf.keras.utils.Sequence): Generator danych testowych.
    """
    
    print(f"Wczytywanie modelu z: {model_path}")
    model = tf.keras.models.load_model(model_path)
    
    # Pobranie nazw klas z generatora (ważne dla raportów)
    # Zakładamy, że generator ma atrybut 'class_indices'
    # Musimy odwrócić mapowanie (indeks -> nazwa klasy)
    class_indices = test_generator.class_indices
    class_names = list(class_indices.keys())
    print(f"Znaleziono {len(class_names)} klas: {class_names}")

    # 1. Pobranie predykcji
    # Używamy model.predict(), aby uzyskać prawdopodobieństwa dla każdej klasy
    print("Rozpoczynanie predykcji na zbiorze testowym...")
    y_pred_probs = model.predict(test_generator)
    
    # Konwertujemy prawdopodobieństwa na konkretne etykiety (klasę z najwyższym p-stwem)
    y_pred = np.argmax(y_pred_probs, axis=1)
    
    # 2. Pobranie prawdziwych etykiet
    y_true = test_generator.classes
    
    # 3. Obliczenie podstawowych metryk
    print("\n--- Raport Ewaluacji Modelu ---")
    
    accuracy = accuracy_score(y_true, y_pred)
    print(f"Ogólna Dokładność (Accuracy): {accuracy * 100:.2f}%")
    
    # 4. Wygenerowanie szczegółowego raportu (Precision, Recall, F1)
    print("\nSzczegółowy Raport Klasyfikacji (sklearn):")
    # target_names pozwala ładnie wyświetlić nazwy klas
    print(classification_report(y_true, y_pred, target_names=class_names))

    # 5. Wygenerowanie macierzy pomyłek
    print("Generowanie macierzy pomyłek...")
    plot_confusion_matrix(y_true, y_pred, class_names)
    
    # TIP: Możesz też policzyć waszą metrykę "Failure-to-Enroll"
    # To byłby osobny test, w którym najpierw detektor (MTCNN) próbuje znaleźć twarz.
    # FTE = (Liczba zdjęć, na których detektor NIE znalazł twarzy) / (Całkowita liczba zdjęć)
    # To jest metryka dla DETEKTORA, nie dla klasyfikatora, ale ważna dla całego systemu.

if __name__ == '__main__':
    # To jest tylko przykład, jak to można uruchomić
    # Będziecie musieli mieć generator danych testowych (np. z `ImageDataGenerator`)
    # i zapisany model.
    
    # Przykładowa ścieżka do modelu
    MODEL_FILE = 'sciezka/do/twojego/modelu.h5' 
    
    # PRZYKŁAD: Musisz tu wstawić swój prawdziwy generator danych testowych
    # test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    # test_generator = test_datagen.flow_from_directory(
    #     'sciezka/do/folderu/testowego',
    #     target_size=(224, 224),
    #     batch_size=32,
    #     class_mode='categorical',
    #     shuffle=False  # Ważne dla ewaluacji! Nie tasuj danych testowych.
    # )

    # Odkomentuj, gdy będziesz gotowy:
    # evaluate_model(MODEL_FILE, test_generator)
    print("Szkielet skryptu ewaluacji gotowy. Zastąp przykładowe dane prawdziwym generatorem.")
